In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [6]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [7]:
base_model=EfficientNetV2M(
    weights='imagenet',
    include_top=False, 
    input_shape=(64,64,3) 
)

214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
base_model.trainable=False

In [9]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-m (Functional)        │ (None, 2, 2, 1280)          │      53,150,388 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,151,128 (210.38 MB)

 Trainable params: 1,997,156 (7.62 MB)

 Non-trainable params: 53,153,972 (202.77 MB)

In [10]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [11]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [12]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [13]:
import time
start_time=time.time()

In [14]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [15]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 157s 119ms/step - accuracy: 0.0163 - loss: 5.2790 - val_accuracy: 0.1216 - val_loss: 4.1078
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 101s 75ms/step - accuracy: 0.0575 - loss: 4.5368 - val_accuracy: 0.2031 - val_loss: 3.5185
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.0991 - loss: 4.0978 - val_accuracy: 0.2693 - val_loss: 3.1256
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.1320 - loss: 3.8280 - val_accuracy: 0.2794 - val_loss: 2.9816
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.1557 - loss: 3.6249 - val_accuracy: 0.3022 - val_loss: 2.8028
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.1795 - loss: 3.4989 - val_accuracy: 0.3192 - val_loss: 2.7035
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.1872 - loss: 3.4069 - val_accuracy: 0.3102 - val_loss: 2.7551
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 90s 77ms/step - accuracy: 0.199

In [16]:
head_training_time=time.time()-start_time

In [17]:
base_model.trainable=True

In [18]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [19]:
start_fine_tune=time.time()

In [20]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 384s 269ms/step - accuracy: 0.1079 - loss: 4.1994 - val_accuracy: 0.2930 - val_loss: 2.7801
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 223s 147ms/step - accuracy: 0.2331 - loss: 3.0957 - val_accuracy: 0.3894 - val_loss: 2.2971
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 173s 148ms/step - accuracy: 0.3032 - loss: 2.7073 - val_accuracy: 0.4490 - val_loss: 2.0303
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 174s 149ms/step - accuracy: 0.3551 - loss: 2.4557 - val_accuracy: 0.4878 - val_loss: 1.8550
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 173s 148ms/step - accuracy: 0.4003 - loss: 2.2478 - val_accuracy: 0.5288 - val_loss: 1.6968
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 173s 148ms/step - accuracy: 0.4356 - loss: 2.1035 - val_accuracy: 0.5601 - val_loss: 1.5616
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 171s 146ms/step - accuracy: 0.4677 - loss: 1.9633 - val_accuracy: 0.5776 - val_loss: 1.4951
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 174s 148ms/step - ac

In [21]:
fine_tune_time=time.time()-start_fine_tune

In [22]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 59ms/step - accuracy: 0.6361 - loss: 1.2667

Test Accuracy: 63.86%
Total training time: 3877.85 seconds


In [23]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.6385 - loss: 1.2702
Validation accuracy: 64.0560%


In [24]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step


In [25]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for EfficientNetV2M:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for EfficientNetV2M:
Accuracy: 0.6386
Precision: 0.6391
Recall: 0.6386
F1-Score: 0.6310


In [26]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["EfficientNetV2M"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [27]:
print(results)

             Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  EfficientNetV2M         0.6386              0.64056   1.258978   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0              1880.07602           1997.772364      55151128  


In [28]:
results.to_csv("EfficientNetV2M.csv",index=False)
print("\nResults saved to EfficientNetV2M.csv")


Results saved to EfficientNetV2M.csv
